In [11]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from collections import namedtuple

# [Test Driving a Headless Browser](https://realpython.com/modern-web-automation-with-python-and-selenium/)

In [2]:
opts = Options()
opts.set_headless()
assert opts.headless  # Operating in headless mode
browser = Firefox(options=opts)
browser.get('https://duckduckgo.com')

/tmp/ipykernel_3950/2639734357.py:2: DeprecationWarning: use setter for headless property instead of set_headless
  opts.set_headless()


In [3]:
search_form = browser.find_element_by_id('search_form_input_homepage')
search_form.send_keys('real python')
search_form.submit()

In [4]:
results = browser.find_elements_by_class_name('result')
print(results[0].text)

Python Tutorials - Real Python
https://realpython.com
Real Python Tutorials. Representing Rational Numbers With Python Fractions. In this tutorial, you'll learn about the Fraction data type in Python, which can represent rational numbers precisely without the rounding errors in binary arithmetic. You'll find that this is especially important in financial and other high-precision applications.


In [5]:
browser.close()

# [Groovin' on Tunes](https://realpython.com/modern-web-automation-with-python-and-selenium/#groovin-on-tunes)

In [6]:
opts = Options()
opts.set_headless()
browser = Firefox(options=opts)
browser.get('https://bandcamp.com')
browser.find_element_by_class_name('playbutton').click()

/tmp/ipykernel_3950/2003952430.py:2: DeprecationWarning: use setter for headless property instead of set_headless
  opts.set_headless()


In [7]:
tracks = browser.find_elements_by_class_name('discover-item')
len(tracks)  #8

8

In [8]:
tracks[3].click()

# Exploring the Catalogue

In [13]:
next_button = [e for e in browser.find_elements_by_class_name('item-page')
                   if e.text.lower().find('next') > -1]

In [14]:
next_button

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="6c8805f3-e957-4406-89c7-69ac53788f92", element="5d5da66b-f516-4f77-a2e6-1d0665659e59")>]

In [16]:
next_button[0].click()

In [17]:
tracks = browser.find_elements_by_class_name('discover-item')
len(tracks) # 

16

You find that your list contains some tracks that were displayed before. To get only the tracks that are actually visible in the browser, you need to filter the results a little.

In [23]:
discover_section = browser.find_element_by_class_name('discover-results')
left_x = discover_section.location['x']
right_x = left_x + discover_section.size['width']
discover_items = browser.find_elements_by_class_name('discover-item')
tracks = [t for t in discover_items
              if t.location['x'] >= left_x and t.location['x'] < right_x]
len(tracks)

8

# Building a Class

In [12]:
TrackRec = namedtuple('TrackRec', [
    'title', 
    'artist',
    'artist_url', 
    'album',
    'album_url', 
    'timestamp'  # When you played it
])

In [13]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from time import sleep, ctime
from collections import namedtuple
from threading import Thread
from os.path import isfile
import csv


BANDCAMP_FRONTPAGE='https://bandcamp.com/'

class BandLeader():
    def __init__(self, csvpath=None):
        self.database_path = csvpath
        self.database = []
        
        # Load database from disk if possible
        if isfile(self.database_path):
            with open(self.database_path, newline='') as dbfile:
                dbreader = csv.reader(dbfile)
                next(dbreader)   # To ignore the header line
                self.database = [TrackRec._make(rec) for rec in dbreader]
                
        # Create a headless browser
        opts = Options()
        opts.set_headless()     
        self.browser = Firefox(options=opts)
        self.browser.get(BANDCAMP_FRONTPAGE)

        # Track list related state
        self._current_track_number = 1
        self.track_list = []
        
        # State for the database
        self._current_track_record = None
        
        # The database maintenance thread
        self.thread = Thread(target=self._maintain)
        self.thread.daemon = True    # kill the thread with the main process dies
        self.thread.start()
        
        self.tracks()
    
    def _maintain(self):
        while True:
            self._update_db()
            sleep(20)    # Check every 20 seconds


    def save_db(self):
        with open(self.database_path, 'w', newline='') as dbfile:
            dbwriter = csv.writer(dbfile)
            dbwriter.writerow(list(TrackRec._fields))
            for entry in self.database:
                dbwriter.writerow(list(entry))
    

    def _update_db(self):
        try:
            check = (self._current_track_record is not None
                     and (len(self.database) == 0 or self.database[-1] != self._current_track_record)
                     and self.is_playing())
            if check:
                self.database.append(self._current_track_record)
                self.save_db()
        
        except Exception as e:
            print('error while updating the db: {}'.format(e))

    def tracks(self):
        '''
        Query the page to populate a list of available tracks.
        '''
        # Sleep to give the browser time to render and finish any animations
        sleep(1)

        # Get the container for the visible track list
        discover_section = self.browser.find_element_by_class_name('discover-results')
        left_x = discover_section.location['x']
        right_x = left_x + discover_section.size['width']

        # Filter the items in the list to include only those we can click
        discover_items = self.browser.find_elements_by_class_name('discover-item')
        self.track_list = [t for t in discover_items
                           if t.location['x'] >= left_x and t.location['x'] < right_x]

        # Print the available tracks to the screen
        for (i,track) in enumerate(self.track_list):
            print('[{}]'.format(i+1))
            lines = track.text.split('\n')
            print('Album  : {}'.format(lines[0]))
            print('Artist : {}'.format(lines[1]))
            if len(lines) > 2:
                print('Genre  : {}'.format(lines[2]))

    def catalogue_pages(self):
        '''
        Print the available pages in the catalogue that are presently accessible.
        '''
        print('PAGES')
        for e in self.browser.find_elements_by_class_name('item-page'):
            print(e.text)
        print('')


    def more_tracks(self, page='next'):
        '''
        Advances the catalogue and repopulates the track list. We can pass in a number
        to advance any of the available pages.
        '''
        next_btn = [e for e in self.browser.find_elements_by_class_name('item-page')
                    if e.text.lower().strip() == str(page)]

        if next_btn:
            next_btn[0].click()
            self.tracks()

    def play(self,track=None):
        '''
        Play a track. If no track number is supplied, the presently selected track will play.
        '''
        if track is None:
            self.browser.find_element_by_class_name('playbutton').click()
        elif type(track) is int and track <= len(self.track_list) and track >= 1:
            self._current_track_number = track
            self.track_list[self._current_track_number - 1].click()
        
        sleep(0.5)
        if self.is_playing():
            self._current_track_record = self.currently_playing()


    def play_next(self):
        '''
        Plays the next available track
        '''
        if self._current_track_number < len(self.track_list):
            self.play(self._current_track_number+1)
        else:
            self.more_tracks()
            self.play(1)


    def pause(self):
        '''
        Pauses the playback
        '''
        self.play()
    

    def is_playing(self):
        '''
        Returns `True` if a track is presently playing
        '''
        playbtn = self.browser.find_element_by_class_name('playbutton')
        return playbtn.get_attribute('class').find('playing') > -1


    def currently_playing(self):
        '''
        Returns the record for the currently playing track,
        or None if nothing is playing
        '''
        try:
            if self.is_playing():
                title = self.browser.find_element_by_class_name('title').text
                album_detail = self.browser.find_element_by_css_selector('.detail-album > a')
                album_title = album_detail.text
                album_url = album_detail.get_attribute('href').split('?')[0]
                artist_detail = self.browser.find_element_by_css_selector('.detail-artist > a')
                artist = artist_detail.text
                artist_url = artist_detail.get_attribute('href').split('?')[0]
                return TrackRec(title, artist, artist_url, album_title, album_url, ctime())

        except Exception as e:
            print('there was an error: {}'.format(e))

        return None